First test to see which LLM to use

Options:
- gpt3-turbo
- LLama
- gpt4.1-nano
- gpt4.1

In [2]:
!pip install openai
!pip install --upgrade PyMuPDF
from API_KEY import API_KEY
from LLAMA_KEY import LLAMA_KEY
import os
import re
import fitz
import json
from openai import OpenAI

output_folder = "limitations_2024"

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

inline_limitations = re.compile(
    r"\b(Limitations?|Challenges?)[:\s]+(.{100,1500}?)",
    flags=re.IGNORECASE | re.DOTALL
)

find_limitations = re.compile(
    r"(?:^|\n)(?:\d{0,2}[\.]\s*)?"
    r"(limitations|limitation|conclusions and limitations|future work|conclusion (?:and|&) future work|limitations (?:and|&) future work|conclusion|conclusions|discussion|conclusion (?:and|&) discussion|conclusions|research limitations|study limitations|challenges)"
    r"(?::)?\s*\n+(.*?)(?=\n\s*(?:\d{1,2}[\.]+\s*)?[A-Z][A-Za-z0-9, \-]{3,60}\n|\Z)",
    flags=re.IGNORECASE | re.DOTALL
)

keywords = ["limitations", "conclusions and limitations","future work", "challenges", "limitation", "study limitations", "research limitations", "limitations and future work", "conclusion and future work", "conclusion & future work"]
def find_section(txt):
    paper_sections = {}
    for f in find_limitations.finditer(txt):
        title = f.group(1).strip().lower()
        text = f.group(2).strip()
        if not text.lower().startswith("question: does the paper discuss the limitations"):
            paper_sections[title] = text
    has_limitations = any(k in paper_sections for k in keywords)
    if not has_limitations:
        for section_title in ["conclusion", "conclusions and limitations" "conclusions", "discussion", "conclusion and discussion", "conclusion & discussion"]:
            section_text = paper_sections.get(section_title)
            if section_text:
                match = inline_limitations.search(section_text)
                if match:
                    paper_sections["inline"] = match.group(1).strip()
                break
    return paper_sections
    
def extract_limitations(path):
    doc = fitz.open(path)
    txt = ""
    for page_num in range(doc.page_count):
        page = doc.load_page(page_num)
        txt += page.get_text("text") + "\n"
    title_paper = os.path.basename(path)
    paper_sections = find_section(txt)
    return title_paper, paper_sections

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 680.4/680.4 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 40.9 MB/s eta 0:00:0000:010:01


In [4]:
client = OpenAI(api_key = API_KEY)
def bullet_list(text, model):
    response = client.chat.completions.create(
        model = model,
        messages = [
            {"role": "system", "content":f"You are a research assistant that summarizes academic limitation sections."},
            {"role": "user", "content":f"Convert the following text into 3-6 clear bullet points:\n {text}"}
        ],
        max_tokens=300
    )
    return (response.choices[0].message.content).strip()
    

In [7]:
!pip install transformers accelerate sentencepiece
!pip install bitsandbytes
from transformers import pipeline
from huggingface_hub import login

login(token=LLAMA_KEY)
llama_pipeline = pipeline("text-generation", model = "meta-llama/Llama-3.2-3B-Instruct",
                   torch_dtype="auto", device_map="auto")

def bullet_list_llama(text):
    prompt = f"""### Instruction:
    You are a research assistant that summarizes academic limitation sections into 3-6 clear bullet points.
    ### Input: {text}
    ### Output: -"""

    
    output = llama_pipeline(prompt, max_new_tokens=300, do_sample=True, top_p=0.95, temperature = 0.7)
    generated = output[0]["generated_text"]
    if '### Output:' in generated:
        return generated.split("### Output:")[-1].strip()
    elif "- " in generated:
        return generated[generated.find("- "):].strip()
    else:
        return generated.strip()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 28.3 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.7/792.7 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 MB 86.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 98.0 MB/s eta 0:00:00:00:0100:01


config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Device set to use cuda:0


In [8]:
year = 2024
full_papers_folder = f"Papers"

papers = [paper for paper in os.listdir(full_papers_folder) if paper.endswith(".pdf")]
limitations_gpt3 = []
limitations_gpt4_nano = []
limitations_gpt4 = []
limitations_llama = []

for paper in papers:
    path = os.path.join(full_papers_folder, paper)
    name, sections = extract_limitations(path)
    for l in keywords:
        section_text = sections.get(l)
        if section_text and section_text.strip().lower() != "question: does the paper discuss the limitations" and ("conclusions and limitations" in sections or "conclusions" in sections or "conclusion" in sections or "discussion" in sections or "conclusion and discussion" in sections or "conclusion & discussion" in sections):
            # Generate bullet points
            bullets_gpt3_turbo = bullet_list(section_text, "gpt-3.5-turbo")
            bullets_gpt4_nano = bullet_list(section_text, "gpt-4.1-nano")
            bullets_gpt4 = bullet_list(section_text, "gpt-4.1")
            bullets_llama = bullet_list_llama(section_text)
            
            
            limitations_gpt3.append({"paper": name, "target": section_text, "target_bullets": bullets_gpt3_turbo})
            limitations_gpt4_nano.append({"paper": name, "target": section_text, "target_bullets": bullets_gpt4_nano})
            limitations_gpt4.append({"paper": name, "target": section_text, "target_bullets": bullets_gpt4})
            limitations_llama.append({"paper": name, "target": section_text, "target_bullets": bullets_llama})
            break
output_file1 = os.path.join(output_folder, f"limitations_only_gpt3_turbo.json")
with open(output_file1, 'w') as out_file:
    json.dump(limitations_gpt3, out_file, indent=4)
output_file2 = os.path.join(output_folder, f"limitations_only_gpt4_nano.json")
with open(output_file2, 'w') as out_file:
    json.dump(limitations_gpt4_nano, out_file, indent=4)
output_file3 = os.path.join(output_folder, f"limitations_only_gpt4.json")
with open(output_file3, 'w') as out_file:
    json.dump(limitations_gpt4, out_file, indent=4)
output_file4 = os.path.join(output_folder, f"limitations_only_llama.json")
with open(output_file4, 'w') as out_file:
    json.dump(limitations_llama, out_file, indent=4)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [ ]:
# !pip install pandas sentence-transformers

import os
import json
import pandas as pd
from sentence_transformers import SentenceTransformer, util

model_output_folder = "limitations_2024"
model = SentenceTransformer("all-MiniLM-L6-v2")

model_files = {
    "gpt3.5-turbo": "limitations_only_gpt3_turbo.json",
    "gpt4.1-nano": "limitations_only_gpt4_nano.json",
    "gpt4.1": "limitations_only_gpt4.json",
    "meta-llama": "limitations_only_llama.json",
    
}


def evaluate_bullets(bullets):
    lines = [l.strip() for l in bullets.split("\n") if l.strip() and l.startswith("-")]
    bullet_count = len(lines)
    has_duplicates = len(set(lines)) < bullet_count
    avg_length = sum(len(l) for l in lines) / bullet_count if bullet_count > 0 else 0

    # Embedding diversity
    if bullet_count > 1:
        embeddings = model.encode(lines, convert_to_tensor=True)
        sim_matrix = util.pytorch_cos_sim(embeddings, embeddings)
        avg_sim = sim_matrix.mean().item()
        diversity = round(1 - avg_sim, 4) 
    else:
        diversity = 0.0

    return {
        "bullet_count": bullet_count,
        "has_duplicates": has_duplicates,
        "format_ok": all(l.startswith("-") for l in lines),
        "avg_length": round(avg_length, 1),
        "embedding_diversity": diversity
    }

all_results = []

for model_name, filename in model_files.items():
    filepath = os.path.join(model_output_folder, filename)
    with open(filepath, "r", encoding="utf-8") as f:
        data = json.load(f)
        for entry in data:
            bullets = entry.get("target_bullets") or entry.get("bullets") or ""
            paper = entry.get("paper") or entry.get("title", "unknown")
            score = evaluate_bullets(bullets)
            all_results.append({
                "paper": paper,
                "model": model_name,
                **score
            })

df = pd.DataFrame(all_results)
df.to_csv("model_bullet_evaluation.csv", index=False)
print(df.head())